<a href="https://colab.research.google.com/github/harshitneverdebugs/ResuMatchAI/blob/main/Resume_Screening.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/UpdatedResumeDataSet.csv")

In [ ]:
df

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
df['Category'].value_counts()

In [ ]:
import re

In [ ]:
import re
def cleanResume(txt):
    cleanText = re.sub('http\S+\s', ' ', txt)
    cleanText = re.sub('RT|cc', ' ', cleanText)
    cleanText = re.sub('#\S+\s', ' ', cleanText)
    cleanText = re.sub('@\S+', '  ', cleanText)
    cleanText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', cleanText)
    cleanText = re.sub(r'[^\x00-\x7f]', ' ', cleanText)
    cleanText = re.sub('\s+', ' ', cleanText)
    return cleanText

In [ ]:
df['Resume'] = df['Resume'].apply(lambda x: cleanResume(x))

In [ ]:
pip install -U sentence-transformers

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util


keywords = input('Enter the keywords you are looking for in the resumes (comma-separated): ')
keywords_list = [keyword.strip().lower() for keyword in keywords.split(',')]
def contains_keywords(content, keywords):
    return any(keyword in content.lower() for keyword in keywords)
shortlisted_resumes = df[df['Resume'].apply(lambda x: contains_keywords(x, keywords_list))]
if len(shortlisted_resumes) == 0:
    print("No candidate is found with the given keywords.")
else:
    print("Shortlisted resumes based on the keywords:")
    print(shortlisted_resumes)
    model = SentenceTransformer('sentence-transformers/distilbert-multilingual-nli-stsb-quora-ranking')
    resume_embeddings = model.encode(shortlisted_resumes['Resume'].tolist())
    reference_embedding = resume_embeddings[0]
    shortlisted_resumes['Similarity Score'] = [util.pytorch_cos_sim(reference_embedding, emb).item() for emb in resume_embeddings]
    ranked_resumes = shortlisted_resumes.sort_values(by='Similarity Score', ascending=False)
    print("Ranked resumes based on similarity scores:")
    print(ranked_resumes[['Similarity Score', 'Category', 'Resume']])
